In [1]:
import numpy as np

# Suppress scientific notation
np.set_printoptions(suppress=True)

# Generate randomly distributed parameters
params = np.random.uniform(low=-50, high=150, size=20)

# Make sure important values are at the beginning for better debugging
params[0] = params.max() + 1
params[1] = params.min() - 1
params[2] = 0

# Round each number to the second decimal place
params = np.round(params, 2)

# Print the parameters
print(params)

[130.19  -7.43   0.   104.11  37.47  20.85 123.78  30.07  -6.43  63.43
  80.28  28.81 129.19  39.87   3.91   1.72  -1.84   2.86  35.57  18.18]


# Assymtric Quantization

In [3]:
def clamp(params_q: np.array, lower_bound: int, upper_bound: int) -> np.array:
    params_q[params_q < lower_bound] = lower_bound
    params_q[params_q > upper_bound] = upper_bound
    return params_q

def asymmetric_quantization(params: np.array, bits: int) -> tuple[np.array, float, int]:
    # Calculate the scale and zero point
    alpha = np.max(params)
    beta = np.min(params)
    scale = (alpha - beta) / (2**bits-1)
    zero = -1*np.round(beta / scale)
    lower_bound, upper_bound = 0, 2**bits-1
    # Quantize the parameters
    quantized = clamp(np.round(params / scale + zero), lower_bound, upper_bound).astype(np.int32)
    return quantized, scale, zero

def asymmetric_dequantize(params_q: np.array, scale: float, zero: int) -> np.array:
    return (params_q - zero) * scale

# Symmetric Quantization

In [2]:
def symmetric_dequantize(params_q: np.array, scale: float) -> np.array:
    return params_q * scale

def symmetric_quantization(params: np.array, bits: int) -> tuple[np.array, float]:
    # Calculate the scale
    alpha = np.max(np.abs(params))
    scale = alpha / (2**(bits-1)-1)
    lower_bound = -2**(bits-1)
    upper_bound = 2**(bits-1)-1
    # Quantize the parameters
    quantized = clamp(np.round(params / scale), lower_bound, upper_bound).astype(np.int32)
    return quantized, scale

In [4]:
def quantization_error(params: np.array, params_q: np.array):
    # calculate the MSE
    return np.mean((params - params_q)**2)

In [5]:
(asymmetric_q, asymmetric_scale, asymmetric_zero) = asymmetric_quantization(params, 8)
(symmetric_q, symmetric_scale) = symmetric_quantization(params, 8)

In [6]:
print(f'Original:')
print(np.round(params, 2))
print('')
print(f'Asymmetric scale: {asymmetric_scale}, zero: {asymmetric_zero}')
print(asymmetric_q)
print('')
print(f'Symmetric scale: {symmetric_scale}')
print(symmetric_q)

Original:
[130.19  -7.43   0.   104.11  37.47  20.85 123.78  30.07  -6.43  63.43
  80.28  28.81 129.19  39.87   3.91   1.72  -1.84   2.86  35.57  18.18]

Asymmetric scale: 0.5396862745098039, zero: 14.0
[255   0  14 207  83  53 243  70   2 132 163  67 253  88  21  17  11  19
  80  48]

Symmetric scale: 1.0251181102362203
[127  -7   0 102  37  20 121  29  -6  62  78  28 126  39   4   2  -2   3
  35  18]


In [7]:
# Dequantize the parameters back to 32 bits
params_deq_asymmetric = asymmetric_dequantize(asymmetric_q, asymmetric_scale, asymmetric_zero)
params_deq_symmetric = symmetric_dequantize(symmetric_q, symmetric_scale)

print(f'Original:')
print(np.round(params, 2))
print('')
print(f'Dequantize Asymmetric:')
print(np.round(params_deq_asymmetric,2))
print('')
print(f'Dequantize Symmetric:')
print(np.round(params_deq_symmetric, 2))

Original:
[130.19  -7.43   0.   104.11  37.47  20.85 123.78  30.07  -6.43  63.43
  80.28  28.81 129.19  39.87   3.91   1.72  -1.84   2.86  35.57  18.18]

Dequantize Asymmetric:
[130.06  -7.56   0.   104.16  37.24  21.05 123.59  30.22  -6.48  63.68
  80.41  28.6  128.99  39.94   3.78   1.62  -1.62   2.7   35.62  18.35]

Dequantize Symmetric:
[130.19  -7.18   0.   104.56  37.93  20.5  124.04  29.73  -6.15  63.56
  79.96  28.7  129.16  39.98   4.1    2.05  -2.05   3.08  35.88  18.45]


In [8]:
# Calculate the quantization error
print(f'{"Asymmetric error: ":>20}{np.round(quantization_error(params, params_deq_asymmetric), 2)}')
print(f'{"Symmetric error: ":>20}{np.round(quantization_error(params, params_deq_symmetric), 2)}')

  Asymmetric error: 0.02
   Symmetric error: 0.07
